* 網址：https://www.kaggle.com/sircausticmail/mod-collaborative-filtering-by-anokas-0-0236446/output

* 修改anokas的模型，Collaborative Filtering based model。

1. It builds a logistic models using the existing products that a user has bought in the 'last month', (and no other attributes) and creates one model for all the products in the 'upcoming month'.

2. Removes all the products bought in the last month.

3. Sorts the products in order of probabilities



In [41]:
import pandas as pd, numpy as np, datetime, gc, re

print(datetime.datetime.now())

usecols = ['ncodpers', 'fecha_dato', 
           'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
           'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
           'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
            'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']


df_train = pd.read_csv('train_ver2.csv', usecols=usecols, parse_dates=['fecha_dato'], infer_datetime_format=True)
# parse_dates對時間欄位做排序
df_train1 = pd.read_csv('train_ver2.csv', usecols=usecols, parse_dates=['fecha_dato'], infer_datetime_format=True)
# parse_dates對時間欄位做排序

2018-03-17 08:22:19.104404


In [42]:
df_train

,fecha_dato,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
5,2015-01-28,1050615,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
6,2015-01-28,1050616,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
7,2015-01-28,1050617,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
8,2015-01-28,1050619,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
9,2015-01-28,1050620,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [3]:
# 只挑4,5月的資料
dftr_prev = df_train[df_train['fecha_dato'].isin(['28-04-2016'])]
dftr_curr = df_train[df_train['fecha_dato'].isin(['28-05-2016'])]

In [5]:
# 去掉時間欄位
dftr_prev.drop(['fecha_dato'], axis=1, inplace = True)
dftr_curr.drop(['fecha_dato'], axis=1, inplace = True)

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
# 把同個顧客4,5月的資料合併，drtr_curr的欄位加上, dftr_prev的欄位加上_prev
dfm = pd.merge(dftr_curr,dftr_prev, how='inner', on=['ncodpers'], suffixes=('', '_prev'))

In [18]:
dfm

,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,...,ind_hip_fin_ult1_prev,ind_plan_fin_ult1_prev,ind_pres_fin_ult1_prev,ind_reca_fin_ult1_prev,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev
0,657640,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,657788,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,657795,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,657790,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1.0,1.0,1
4,657794,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
5,657797,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
6,657789,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0.0,0.0,0
7,657787,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
8,657777,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
9,657782,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [10]:
prevcols = [col for col in dfm.columns if '_ult1_prev' in col]
currcols = [col for col in dfm.columns if '_ult1' in col and '_ult1_prev' not in col]

In [13]:
prevcols

['ind_ahor_fin_ult1_prev',
 'ind_aval_fin_ult1_prev',
 'ind_cco_fin_ult1_prev',
 'ind_cder_fin_ult1_prev',
 'ind_cno_fin_ult1_prev',
 'ind_ctju_fin_ult1_prev',
 'ind_ctma_fin_ult1_prev',
 'ind_ctop_fin_ult1_prev',
 'ind_ctpp_fin_ult1_prev',
 'ind_deco_fin_ult1_prev',
 'ind_deme_fin_ult1_prev',
 'ind_dela_fin_ult1_prev',
 'ind_ecue_fin_ult1_prev',
 'ind_fond_fin_ult1_prev',
 'ind_hip_fin_ult1_prev',
 'ind_plan_fin_ult1_prev',
 'ind_pres_fin_ult1_prev',
 'ind_reca_fin_ult1_prev',
 'ind_tjcr_fin_ult1_prev',
 'ind_valo_fin_ult1_prev',
 'ind_viv_fin_ult1_prev',
 'ind_nomina_ult1_prev',
 'ind_nom_pens_ult1_prev',
 'ind_recibo_ult1_prev']

In [14]:
currcols

['ind_ahor_fin_ult1',
 'ind_aval_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1']

In [15]:
# 型態設為category
for cols in currcols:
    print(cols)
    dfm[cols] = dfm[cols].astype('category')

for cols in prevcols:
    print(cols)
    dfm[cols] = dfm[cols].astype('category')

ind_ahor_fin_ult1
ind_aval_fin_ult1
ind_cco_fin_ult1
ind_cder_fin_ult1
ind_cno_fin_ult1
ind_ctju_fin_ult1
ind_ctma_fin_ult1
ind_ctop_fin_ult1
ind_ctpp_fin_ult1
ind_deco_fin_ult1
ind_deme_fin_ult1
ind_dela_fin_ult1
ind_ecue_fin_ult1
ind_fond_fin_ult1
ind_hip_fin_ult1
ind_plan_fin_ult1
ind_pres_fin_ult1
ind_reca_fin_ult1
ind_tjcr_fin_ult1
ind_valo_fin_ult1
ind_viv_fin_ult1
ind_nomina_ult1
ind_nom_pens_ult1
ind_recibo_ult1
ind_ahor_fin_ult1_prev
ind_aval_fin_ult1_prev
ind_cco_fin_ult1_prev
ind_cder_fin_ult1_prev
ind_cno_fin_ult1_prev
ind_ctju_fin_ult1_prev
ind_ctma_fin_ult1_prev
ind_ctop_fin_ult1_prev
ind_ctpp_fin_ult1_prev
ind_deco_fin_ult1_prev
ind_deme_fin_ult1_prev
ind_dela_fin_ult1_prev
ind_ecue_fin_ult1_prev
ind_fond_fin_ult1_prev
ind_hip_fin_ult1_prev
ind_plan_fin_ult1_prev
ind_pres_fin_ult1_prev
ind_reca_fin_ult1_prev
ind_tjcr_fin_ult1_prev
ind_valo_fin_ult1_prev
ind_viv_fin_ult1_prev
ind_nomina_ult1_prev
ind_nom_pens_ult1_prev
ind_recibo_ult1_prev


In [16]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

In [17]:
dfm.fillna(0, inplace=True)
#NaN以0取代
#對於不知道就當作沒有持有產品 補零

In [19]:
# 準備相關變數
models = {}
model_preds = {}
id_preds = defaultdict(list)

model_preds_train = {}
id_preds_train = defaultdict(list)

In [20]:
ids = dfm['ncodpers'].values
testids = dftr_curr['ncodpers'].values

In [25]:
ids

array([ 657640,  657788,  657795, ..., 1166764, 1166763, 1166789])

In [26]:
testids

array([ 657640,  657788,  657795, ..., 1166763, 1166789, 1550586])

In [27]:
prevcols = [col for col in dfm.columns if '_ult1_prev' in col]
currcols = [col for col in dfm.columns if '_ult1' in col and '_ult1_prev' not in col]

In [29]:
for c in dfm.columns:
    if c != 'ncodpers' and '_ult1_prev' not in c:
        print(c)
        # 5月產品
        y_train = dfm[c]
        # 去掉時間在五月的欄位，拿4月train>>>>>4月擁有產品去看5月
        x_train = dfm.drop(currcols,1,inplace=False).drop(['ncodpers'],1,inplace=False)
        # 使用5月資料test
        x_test = dftr_curr.drop(['ncodpers'],1,inplace=False)
        
        clf = LogisticRegression()
        clf.fit(x_train, y_train)
        p_train = clf.predict_proba(x_train)[:,1]
        # 相較於上一篇多了p_test做測試
        p_test = clf.predict_proba(x_test)[:,1]
        
        models[c] = clf # 產品：模型
        model_preds[c] = p_test # 產品：測試機率
        for id, p in zip(testids, p_test):
            id_preds[id].append(p)
            
        model_preds_train[c] = p_train #產品：訓練機率
        for id, p in zip(ids, p_train):
            id_preds_train[id].append(p)
            
        print(roc_auc_score(y_train, p_train))

ind_ahor_fin_ult1
0.999933191401
ind_aval_fin_ult1
1.0
ind_cco_fin_ult1
0.990331553123
ind_cder_fin_ult1
0.995716706099
ind_cno_fin_ult1
0.993427749136
ind_ctju_fin_ult1
0.999276945815
ind_ctma_fin_ult1
0.972760351706
ind_ctop_fin_ult1
0.998786439569
ind_ctpp_fin_ult1
0.998997198337
ind_deco_fin_ult1
0.999998200249
ind_deme_fin_ult1
0.99999507079
ind_dela_fin_ult1
0.998783715597
ind_ecue_fin_ult1
0.994866530873
ind_fond_fin_ult1
0.999108398666
ind_hip_fin_ult1
0.999959925059
ind_plan_fin_ult1
0.999370354704
ind_pres_fin_ult1
0.999332056036
ind_reca_fin_ult1
0.998782684681
ind_tjcr_fin_ult1
0.983727685671
ind_valo_fin_ult1
0.998401118874
ind_viv_fin_ult1
0.99991146753
ind_nomina_ult1
0.987102596298
ind_nom_pens_ult1
0.98766174203
ind_recibo_ult1
0.97705268851


In [30]:
already_active = {}

dftrcurr = dftr_curr

for row in dftrcurr.values:
    row = list(row)
    #移除list中給定位置的項目>>>>移除ncodpers
    id = row.pop(0)
        
    #選出每個顧客已經擁有的產品    
    active = [c[0] for c in zip(dftrcurr.columns[1:], row) if c[1] > 0]
    already_active[id] = active

In [31]:
test_preds = {}
for id, p in id_preds.items():
    # 挑選每位顧客前7名的產品，並排除顧客已經擁有的產品
    preds = [i[0] for i in sorted([i for i in zip(dftrcurr.columns[1:], p) if i[0] not in already_active[id]], key=lambda i:i [1], reverse=True)[:7]]
    test_preds[id] = preds

In [33]:
sample = pd.read_csv('sample_submission.csv')

In [ ]:
# 找出train好的顧客，並處理成等等要匯出csv的樣子
test_preds_prods = []
for row in sample.values:
    id = row[0]
    p = test_preds[id]
    test_preds_prods.append(' '.join(p))

sample['added_products'] = test_preds_prods
sample.to_csv('modified_collab_sub_by_KB.csv', index=False)